# Système Multi-Agents avec Q-Learning pour l'Ordonnancement des Patients

Ce notebook démontre l'implémentation d'un système multi-agents pour l'optimisation de l'ordonnancement des patients dans un environnement de soins.

## Caractéristiques principales:
- **Données Strictes**: Basées sur la table de compétences réelle (Image WhatsApp).
- **Métaheuristiques hybrides**: Algorithme Génétique, Recherche Tabou, Recuit Simulé
- **Modes de collaboration**: Amis (partage complet) et Ennemis (compétition)
- **Auto-adaptation**: Q-Learning pour la sélection des voisinages (Insertion vs Swap)

Basé sur le diaporama: *"Optimisation collaborative : Agents auto-adaptatifs, Apprentissage par renforcement"*

In [ ]:
# Configuration et imports
import sys
import os
import random
import numpy as np
import matplotlib.pyplot as plt

# Ajout du chemin racine
sys.path.insert(0, os.path.abspath(os.getcwd()))

# Seed pour reproductibilité
random.seed(42)
np.random.seed(42)

# Imports du projet remasterisé
from core.environment import create_default_environment
from core.neighborhoods import NeighborhoodManager
from core.shared_memory import SharedMemoryPool, Solution
from core.agents import MultiAgentSystem, CollaborationMode
from visualization import plot_gantt

print("✓ Tous les modules sont chargés et prêts!")

## 1. Environnement d'Ordonnancement (Strict)

L'environnement modélise l'ordonnancement des patients selon les contraintes strictes de l'image fournie.

In [ ]:
# Créer l'environnement
env = create_default_environment()

print(f"Configuration de l'environnement:")
print(f"  - Nombre de patients: {env.num_patients}")
print(f"  - Compétences: {env.skills}")
print(f"  - Nombre total de tâches: {len(env.all_tasks)}")

# Solution initiale (Aléatoire)
initial_solution = env.build_initial_solution(random_order=True)
initial_makespan, initial_times, _ = env.evaluate(initial_solution, return_schedule=True)
print(f"\nMakespan initial (Aléatoire): {initial_makespan}")

In [ ]:
# Visualiser le planning initial
if initial_times:
    plot_gantt(
        initial_times, env.skills, env.num_patients,
        title=f"Planning Initial (Cmax = {initial_makespan})"
    )

## 2. Fonctions de Voisinage & Q-Learning

Pour respecter les contraintes strictes (1 compétence spécifique par tâche), nous utilisons uniquement les voisinages valides :
- **C**: Insertion (Déplacer une tâche dans la file du médecin)
- **E**: Swap (Échanger deux tâches chez le même médecin)

Le **Q-Learning** va apprendre lequel des deux est le plus efficace.

In [ ]:
# Démonstration Q-Learning via un Agent Tabou
mas_demo = MultiAgentSystem(env, mode=CollaborationMode.FRIENDS, use_qlearning=True)
agent_tabu = mas_demo.add_agent('tabu', 'Demo_Agent', tabu_tenure=5)

# On force quelques mises à jour pour la démo
if agent_tabu.q_selector:
    print("Simulation de l'apprentissage...")
    # Simulation : L'insertion (C) donne de meilleurs résultats ici
    for _ in range(10):
        agent_tabu.q_selector.update_with_result('C', 100) # Bonne récompense
        agent_tabu.q_selector.update_with_result('E', 110) # Moins bonne récompense (Cmax plus haut)
    
    stats = agent_tabu.q_selector.get_statistics()
    q_table = agent_tabu.q_selector.get_q_table()
    
    print("\nÉtat du Q-Learning :")
    print(f"  Voisinages actifs : {list(stats.keys())}")
    print(f"  Table Q : {q_table}")
else:
    print("Q-Learning désactivé.")

## 3. Système Multi-Agents - Mode AMIS (Collaboratif)

En mode Amis, les agents (AG, Tabou, RS) partagent leurs solutions via un **Espace Mémoire Partagé (EMP)**.

In [ ]:
print("🚀 Lancement SMA Collaboratif (100 itérations)...")

mas_friends = MultiAgentSystem(env, mode=CollaborationMode.FRIENDS, use_qlearning=True)

# Trio complet
mas_friends.add_agent('genetic', 'AG_Main', population_size=15)
mas_friends.add_agent('tabu', 'Tabu_Main', tabu_tenure=10)
mas_friends.add_agent('sa', 'RS_Main', initial_temp=100)

best_sol_friends = mas_friends.run(n_iterations=100, verbose=True)

print(f"\n🏆 Meilleur Makespan (AMIS): {best_sol_friends.fitness}")

In [ ]:
# Visualisation du résultat final Collaboratif
if best_sol_friends:
    cmax, times, _ = env.evaluate(best_sol_friends.sequences, return_schedule=True)
    plot_gantt(
        times, env.skills, env.num_patients,
        title=f"Résultat SMA Amis (Cmax={cmax})"
    )

## 4. Système Multi-Agents - Mode ENNEMIS (Compétitif)

En mode Ennemis, les agents travaillent isolément et ne partagent pas leur mémoire.

In [ ]:
print("⚔️ Lancement SMA Compétitif (100 itérations)...")

mas_enemies = MultiAgentSystem(env, mode=CollaborationMode.ENEMIES, use_qlearning=True)

mas_enemies.add_agent('genetic', 'AG_Enemy')
mas_enemies.add_agent('tabu', 'Tabu_Enemy')
mas_enemies.add_agent('sa', 'RS_Enemy')

best_sol_enemies = mas_enemies.run(n_iterations=100, verbose=True)

print(f"\n🏆 Meilleur Makespan (ENNEMIS): {best_sol_enemies.fitness}")

## 5. Comparaison et Génération des Tableaux (PDF)

Génération des métriques pour les slides 25 et 26.

In [ ]:
print("="*60)
print("        RÉSULTATS POUR LE RAPPORT (PDF)")
print("="*60)

# Résultats simulés sur la dernière exécution
score_amis = best_sol_friends.fitness
score_ennemis = best_sol_enemies.fitness

print(f"{'Configuration':<25} | {'Makespan (Score)'}")
print("-"*45)
print(f"{'Mode AMIS (Collaboratif)':<25} | {score_amis}")
print(f"{'Mode ENNEMIS (Compétitif)':<25} | {score_ennemis}")

gain = ((score_ennemis - score_amis) / score_ennemis) * 100
print("-"*45)
if gain > 0:
    print(f"✅ La collaboration a amélioré le score de {gain:.2f}%")
else:
    print(f"⚠️ Pas de gain significatif observé sur cette run courte.")

## Conclusion

Le système respecte les contraintes strictes. Le **mode Amis** couplé au **Q-Learning** (choix dynamique entre Insertion et Swap) offre généralement la meilleure convergence en évitant les minimums locaux grâce à l'Espace Mémoire Partagé.